In [1]:
import agents, environments

import gym
from gym import spaces
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import t, mode
from tqdm import tqdm
import cProfile

from utility import print_gridworld_with_policy, print_heatmap, plot_graph

2023-10-20 14:01:09.818349: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-20 14:01:09.906561: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-20 14:01:09.907357: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 14:01:11.633329: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Global Variables

In [2]:
confidence = 0.95

# Register the environment

In [3]:
gym.envs.register(
    id='GridworldPOMDPEnvGoalless-v0',
    entry_point=environments.GridworldPOMDPEnvGoalless
)

# Gridworld Vector First

In [5]:
# Number of episodes for the training
n_episodes = 50
n_traj = 100
# Number of runs per episode
n_run = 2
# Define the arguments for each environment
time_horizon = 25
steepness = 15
prob = 0.1
envs = gym.vector.make('GridworldPOMDPEnvGoalless-v0', time_horizon=time_horizon, steepness=steepness, prob=prob, num_envs=n_traj)
env = environments.GridworldPOMDPEnvGoalless(time_horizon = time_horizon, steepness=15, prob=prob)

In [ ]:
def main():
    with tqdm(total=n_run * n_episodes, ncols=80) as pbar:
        # Train the agent and plot the entropies
        list_entropies = []
        list_true_entropies = []
        for r in range(n_run):
            ag = agents.REINFORCEAgentEPOMDPVec(env, alpha=0.5, n_traj=n_traj)  # Modify the agent to accept the parallel environments
            avg_entropies = []
            avg_true_entropies = []
            for i in range(n_episodes):
                trajectories = []
                episodes = [ [] for _ in range(n_traj) ]
                true_entropies = []
                d_t = np.zeros((n_traj, env.observation_space.n))  # Initialize visitation counts for all parallel environments
                true_d_t = np.zeros((n_traj, env.observation_space.n))
                envs.reset()
                done = np.zeros(n_traj, dtype=bool)
                while not np.all(done):
                    # Sample action and get probabilities from the belief
                    actions, probs = ag.get_actions()
                    # Sample state
                    sampled_states = ag.get_states()
                    # Take a step in the parallel environments
                    next_obs, rewards, done, _, true_states = envs.step(actions)
                    # Get the indices of the states for all parallel environments
                    state_indices = [env.state_to_index(state) for state in sampled_states]
                    true_state_indices = true_states['true_state']
                    # Update state visitation counts for all parallel environments
                    for i, state_index in enumerate(state_indices):
                        d_t[i][state_index] += 1
                    for i, true_state_index in enumerate(true_state_indices):
                        true_d_t[i][true_state_index] += 1
                    # Arrange the single trajectories
                    for i in range(n_traj):
                        episodes[i].append((ag.beliefs[i], actions[i], probs[i], rewards[i], true_state_indices[i]))
                    # Update belief
                    ag.belief_update(actions, next_obs)
                # Compute true entropy of the trajectory for all parallel environments
                true_d_t /= time_horizon
                log_true_d_t = -np.log(true_d_t, where=true_d_t>0)
                true_entropies = np.sum(np.multiply(true_d_t, log_true_d_t),axis=1)
                # Compute believed entropy of the trajectory for all parallel environments
                d_t /= time_horizon
                log_d_t = -np.log(d_t, where=d_t>0)
                entropies = np.sum(np.multiply(d_t, log_d_t), axis=1)
                for i in range(n_traj):
                    trajectories.append((episodes[i], d_t[i]))
                ag.update_multiple_sampling(trajectories)
                avg_entropies.append(np.mean(entropies))
                avg_true_entropies.append(np.mean(true_entropies))
                pbar.update(1)
            ag.print_visuals(envs=envs, env=env, n_traj=n_traj)  # Modify the agent to accept the parallel environments
            list_entropies.append(avg_entropies)
            list_true_entropies.append(avg_true_entropies)
        list_entropies = np.transpose(np.array(list_entropies), (1, 0))
        list_true_entropies = np.transpose(np.array(list_true_entropies), (1, 0))

    plot_graph(n_run, n_episodes, list_entropies, list_true_entropies, confidence, 'Example with parallelism')

#main()

if __name__ == "__main__":
    cProfile.run("main()", sort="cumulative")

In [6]:
def main():
    with tqdm(total=n_run * n_episodes, ncols=80) as pbar:
        # Train the agent and plot the entropies
        list_entropies = []
        list_true_entropies = []
        for r in range(n_run):
            ag = agents.REINFORCEAgentEPOMDP(env, alpha = 0.25)
            avg_entropies = []
            avg_true_entropies = []
            for i in range(n_episodes):
                episodes, true_entropies = ag.play(env=env, n_traj=n_traj)
                entropies = ag.update_multiple_sampling(episodes)
                avg_entropies.append(np.mean(entropies))
                avg_true_entropies.append(np.mean(true_entropies))
                pbar.update(1)
            ag.print_visuals(env=env, n_traj=n_traj)
            list_entropies.append(avg_entropies)
            list_true_entropies.append(avg_true_entropies)
        list_entropies = np.transpose(np.array(list_entropies), (1,0))
        list_true_entropies = np.transpose(np.array(list_true_entropies), (1,0))

    plot_graph(n_run, n_episodes, list_entropies, list_true_entropies, confidence, 'Example sequential')

# main()

if __name__ == "__main__":
    cProfile.run("main()", sort="cumulative")

  0%|                                                   | 0/100 [00:03<?, ?it/s]


         1514132 function calls (1506430 primitive calls) in 3.646 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.650    3.650 {built-in method builtins.exec}
        1    0.000    0.000    3.650    3.650 <string>:1(<module>)
        1    0.000    0.000    3.650    3.650 927008859.py:1(main)
        1    0.032    0.032    3.490    3.490 agents.py:542(play)
     2500    0.026    0.000    2.626    0.001 agents.py:522(get_state)
     2500    0.136    0.000    2.364    0.001 _axis_nan_policy.py:383(axis_nan_policy_wrapper)
68108/60406    0.149    0.000    1.176    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     2500    0.010    0.000    0.784    0.000 _axis_nan_policy.py:14(_broadcast_arrays)
    10000    0.540    0.000    0.723    0.000 {method 'choice' of 'numpy.random.mtrand.RandomState' objects}
     2500    0.009    0.000    0.695    0.000 _axis_nan_pol

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.